In [1]:
import os
import spacy 
import pickle
import random
import nltk
import pandas as pd
import numpy as np
import nltk
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

In [2]:
nlp = spacy.load('en_core_web_sm')
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [504]:
df = pd.read_csv(r'C:\Users\trainee\Downloads\Mohammed Khalaf\job_skills_google dataset.csv')

In [505]:
df

,Company,Title,Category,Location,Responsibilities,Minimum Qualifications,Preferred Qualifications
0,Google,Google Cloud Program Manager,Program Management,Singapore,"Shape, shepherd, ship, and show technical prog...",BA/BS degree or equivalent practical experienc...,Experience in the business technology market a...
1,Google,"Supplier Development Engineer (SDE), Cable/Con...",Manufacturing & Supply Chain,"Shanghai, China",Drive cross-functional activities in the suppl...,BS degree in an Engineering discipline or equi...,"BSEE, BSME or BSIE degree.\nExperience of usin..."
2,Google,"Data Analyst, Product and Tools Operations, Go...",Technical Solutions,"New York, NY, United States",Collect and analyze data to draw insight and i...,"Bachelor’s degree in Business, Economics, Stat...",Experience partnering or consulting cross-func...
3,Google,"Developer Advocate, Partner Engineering",Developer Relations,"Mountain View, CA, United States","Work one-on-one with the top Android, iOS, and...",BA/BS degree in Computer Science or equivalent...,"Experience as a software developer, architect,..."
4,Google,"Program Manager, Audio Visual (AV) Deployments",Program Management,"Sunnyvale, CA, United States",Plan requirements with internal customers.\nPr...,BA/BS degree or equivalent practical experienc...,CTS Certification.\nExperience in the construc...
...,...,...,...,...,...,...,...
1245,Google,Global Investigator,Real Estate & Workplace Services,"Boulder, CO, United States",Develop protocols and practices to assess and ...,BA/BS degree or equivalent practical experienc...,Investigative or Interview certification throu...
1246,Google,Campus Security Manager,Real Estate & Workplace Services,"Boulder, CO, United States",Manage the daily operations of the security ve...,BA/BS degree or equivalent practical experienc...,Master’s degree.\nProficient in the use of com...
1247,Google,Facilities Manager,Real Estate & Workplace Services,"Xinyi District, Taiwan",Collaborate with functional teams; plan and ma...,BA/BS degree or equivalent practical experienc...,Experience managing outsourced facilities serv...
1248,Google,Physical Security Manager,Real Estate & Workplace Services,"Tel Aviv-Yafo, Israel",Partner closely with and offer guidance and co...,BA/BS degree or equivalent practical experienc...,Experience working on international programs.\...


Check the missing values:

In [442]:
index = df['Preferred Qualifications'].index[df['Preferred Qualifications'].isnull()]
index

Int64Index([15, 72, 91, 97, 98, 123, 150, 160, 202, 206, 210, 230, 241, 286], dtype='int64')

In [443]:
df['Preferred Qualifications'][15]

nan

In [444]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250 entries, 0 to 1249
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Company                   1250 non-null   object
 1   Title                     1250 non-null   object
 2   Category                  1250 non-null   object
 3   Location                  1250 non-null   object
 4   Responsibilities          1235 non-null   object
 5   Minimum Qualifications    1236 non-null   object
 6   Preferred Qualifications  1236 non-null   object
dtypes: object(7)
memory usage: 68.5+ KB


In [445]:
df.isnull().sum()

Company                      0
Title                        0
Category                     0
Location                     0
Responsibilities            15
Minimum Qualifications      14
Preferred Qualifications    14
dtype: int64

In [446]:
df.describe()

,Company,Title,Category,Location,Responsibilities,Minimum Qualifications,Preferred Qualifications
count,1250,1250,1250,1250,1235,1236,1236
unique,2,794,23,92,801,807,821
top,Google,Business Intern 2018,Sales & Account Management,"Mountain View, CA, United States",Responsibilities and detailed projects will be...,Must be pursuing a full-time MBA program with ...,"Previous internship experience in Sales, Adver..."
freq,1227,35,168,190,33,32,32


In [447]:
print("Missing Value", ((df.isnull().sum().sum())/df.shape[0])*100)

Missing Value 3.44


In [448]:
myData = df

In [449]:
for i in ['Minimum Qualifications','Preferred Qualifications']:
    myData[i].astype('category')
    print(myData[i].unique())
    myData[i]= myData[i].replace(0, np.nan)

['BA/BS degree or equivalent practical experience.\n3 years of experience in program and/or project management in cloud computing, enterprise software and/or marketing technologies.'
 'BS degree in an Engineering discipline or equivalent practical experience.\n7 years of experience in Cable/Connector Design or Manufacturing in an NPI role.\nExperience working with Interconnect Engineering and Product Design (PD)/Mechanical Engineer in developing, manufacturing and testing.\nAbility to speak and write in English and Mandarin fluently and idiomatically.'
 'Bachelor’s degree in Business, Economics, Statistics, Operations Research or related analytical field, or equivalent practical experience.\n2 years of work experience in business analysis.\n1 year of experience with statistical modeling, forecasting or machine learning. Experience with R, SAS or Python.\n1 year of experience developing and manipulating large datasets.'
 'BA/BS degree in Computer Science or equivalent practical experien

In [450]:
#for i in ['Minimum Qualifications', 'Preferred Qualifications']:
#   most_frequent = myData[i].mode()
#  myData[i].fillna(most_frequent[0], inplace=True)

In [451]:
myData.drop(columns = ['Company' , 'Location', 'Responsibilities'], inplace = True)
myData

,Title,Category,Minimum Qualifications,Preferred Qualifications
0,Google Cloud Program Manager,Program Management,BA/BS degree or equivalent practical experienc...,Experience in the business technology market a...
1,"Supplier Development Engineer (SDE), Cable/Con...",Manufacturing & Supply Chain,BS degree in an Engineering discipline or equi...,"BSEE, BSME or BSIE degree.\nExperience of usin..."
2,"Data Analyst, Product and Tools Operations, Go...",Technical Solutions,"Bachelor’s degree in Business, Economics, Stat...",Experience partnering or consulting cross-func...
3,"Developer Advocate, Partner Engineering",Developer Relations,BA/BS degree in Computer Science or equivalent...,"Experience as a software developer, architect,..."
4,"Program Manager, Audio Visual (AV) Deployments",Program Management,BA/BS degree or equivalent practical experienc...,CTS Certification.\nExperience in the construc...
...,...,...,...,...
1245,Global Investigator,Real Estate & Workplace Services,BA/BS degree or equivalent practical experienc...,Investigative or Interview certification throu...
1246,Campus Security Manager,Real Estate & Workplace Services,BA/BS degree or equivalent practical experienc...,Master’s degree.\nProficient in the use of com...
1247,Facilities Manager,Real Estate & Workplace Services,BA/BS degree or equivalent practical experienc...,Experience managing outsourced facilities serv...
1248,Physical Security Manager,Real Estate & Workplace Services,BA/BS degree or equivalent practical experienc...,Experience working on international programs.\...


In our dataset we have two columns of qulaifications "Minimum Qualifications" and  "Preferred Qualifications"
to get most of mentioned skills

In [452]:
myData['Qualifications'] = myData["Minimum Qualifications"] + myData["Preferred Qualifications"]
myData

,Title,Category,Minimum Qualifications,Preferred Qualifications,Qualifications
0,Google Cloud Program Manager,Program Management,BA/BS degree or equivalent practical experienc...,Experience in the business technology market a...,BA/BS degree or equivalent practical experienc...
1,"Supplier Development Engineer (SDE), Cable/Con...",Manufacturing & Supply Chain,BS degree in an Engineering discipline or equi...,"BSEE, BSME or BSIE degree.\nExperience of usin...",BS degree in an Engineering discipline or equi...
2,"Data Analyst, Product and Tools Operations, Go...",Technical Solutions,"Bachelor’s degree in Business, Economics, Stat...",Experience partnering or consulting cross-func...,"Bachelor’s degree in Business, Economics, Stat..."
3,"Developer Advocate, Partner Engineering",Developer Relations,BA/BS degree in Computer Science or equivalent...,"Experience as a software developer, architect,...",BA/BS degree in Computer Science or equivalent...
4,"Program Manager, Audio Visual (AV) Deployments",Program Management,BA/BS degree or equivalent practical experienc...,CTS Certification.\nExperience in the construc...,BA/BS degree or equivalent practical experienc...
...,...,...,...,...,...
1245,Global Investigator,Real Estate & Workplace Services,BA/BS degree or equivalent practical experienc...,Investigative or Interview certification throu...,BA/BS degree or equivalent practical experienc...
1246,Campus Security Manager,Real Estate & Workplace Services,BA/BS degree or equivalent practical experienc...,Master’s degree.\nProficient in the use of com...,BA/BS degree or equivalent practical experienc...
1247,Facilities Manager,Real Estate & Workplace Services,BA/BS degree or equivalent practical experienc...,Experience managing outsourced facilities serv...,BA/BS degree or equivalent practical experienc...
1248,Physical Security Manager,Real Estate & Workplace Services,BA/BS degree or equivalent practical experienc...,Experience working on international programs.\...,BA/BS degree or equivalent practical experienc...


In [453]:
myData['Qualifications'][15]

nan

In [454]:
nlp = spacy.load('en_core_web_sm')
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [455]:
skill_pattern_path = r'C:\Users\trainee\Downloads\Mohammed Khalaf\Resume\jz_skill_patterns.jsonl'
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [456]:
import re
clean = []
for i in range(myData.shape[0]):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        str(myData["Qualifications"].iloc[i])
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    clean.append(review)

In [457]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))

In [458]:
myData["Qualifications_Cleaned"] = clean
myData["skills"] = myData["Qualifications_Cleaned"].str.lower().apply(get_skills)
myData["skills"] = myData["skills"].apply(unique_skills)
myData.head()

,Title,Category,Minimum Qualifications,Preferred Qualifications,Qualifications,Qualifications_Cleaned,skills
0,Google Cloud Program Manager,Program Management,BA/BS degree or equivalent practical experienc...,Experience in the business technology market a...,BA/BS degree or equivalent practical experienc...,ba b degree equivalent practical experience 3 ...,"[business, software, project management, marke..."
1,"Supplier Development Engineer (SDE), Cable/Con...",Manufacturing & Supply Chain,BS degree in an Engineering discipline or equi...,"BSEE, BSME or BSIE degree.\nExperience of usin...",BS degree in an Engineering discipline or equi...,b degree engineering discipline equivalent pra...,"[design, testing, data analysis, material, his..."
2,"Data Analyst, Product and Tools Operations, Go...",Technical Solutions,"Bachelor’s degree in Business, Economics, Stat...",Experience partnering or consulting cross-func...,"Bachelor’s degree in Business, Economics, Stat...",bachelor degree business economics statistic o...,"[business, query language, database, machine l..."
3,"Developer Advocate, Partner Engineering",Developer Relations,BA/BS degree in Computer Science or equivalent...,"Experience as a software developer, architect,...",BA/BS degree in Computer Science or equivalent...,ba b degree computer science equivalent practi...,"[computer science, android, mobile, software, ..."
4,"Program Manager, Audio Visual (AV) Deployments",Program Management,BA/BS degree or equivalent practical experienc...,CTS Certification.\nExperience in the construc...,BA/BS degree or equivalent practical experienc...,ba b degree equivalent practical experience 4 ...,"[design, business, software, deployment, syste..."


In [434]:
myData['Qualifications_Cleaned'][1245]

'ba b degree equivalent practical experience 4 year experience investigation private sector public sector environment investigative interview certification national international organization 1 year experience conducting diverse investigation ranging workplace violence threat mitigation financial crime corrupt practice theft insider threat 1 year experience supply chain investigation 1 year experience leading interview including non confrontational interviewing technique cctv experience ability speak write spanish chinese portuguese hindi fluently idiomatically'

In [460]:
myData2 = myData

In [461]:
myData2.drop(columns = ['Minimum Qualifications' , 'Preferred Qualifications', 'Qualifications','Qualifications_Cleaned'], inplace = True)
myData2

,Title,Category,skills
0,Google Cloud Program Manager,Program Management,"[business, software, project management, marke..."
1,"Supplier Development Engineer (SDE), Cable/Con...",Manufacturing & Supply Chain,"[design, testing, data analysis, material, his..."
2,"Data Analyst, Product and Tools Operations, Go...",Technical Solutions,"[business, query language, database, machine l..."
3,"Developer Advocate, Partner Engineering",Developer Relations,"[computer science, android, mobile, software, ..."
4,"Program Manager, Audio Visual (AV) Deployments",Program Management,"[design, business, software, deployment, syste..."
...,...,...,...
1245,Global Investigator,Real Estate & Workplace Services,[]
1246,Campus Security Manager,Real Estate & Workplace Services,"[business, access control, security, software]"
1247,Facilities Manager,Real Estate & Workplace Services,[]
1248,Physical Security Manager,Real Estate & Workplace Services,[security]


In [462]:
size = len(myData2['skills'])
size

1250

In [493]:
myData2['Title'][100]

'ソフトウェア エンジニア、Chrome OS ビデオ'

In [463]:
myData2=myData2.explode('skills')
myData2

,Title,Category,skills
0,Google Cloud Program Manager,Program Management,business
0,Google Cloud Program Manager,Program Management,software
0,Google Cloud Program Manager,Program Management,project management
0,Google Cloud Program Manager,Program Management,marketing
0,Google Cloud Program Manager,Program Management,engineering
...,...,...,...
1246,Campus Security Manager,Real Estate & Workplace Services,security
1246,Campus Security Manager,Real Estate & Workplace Services,software
1247,Facilities Manager,Real Estate & Workplace Services,NaN
1248,Physical Security Manager,Real Estate & Workplace Services,security


In [479]:
myData2["skills"].fillna("None", inplace = True)

In [480]:
myData2.isnull().sum()

Title       0
Category    0
skills      0
dtype: int64

In [483]:
out = myData2.to_csv(r'C:\Users\trainee\Downloads\Mohammed Khalaf\Dataset.csv',index = False)

In [491]:
myData2

,Title,Category,skills
0,Google Cloud Program Manager,Program Management,business
0,Google Cloud Program Manager,Program Management,software
0,Google Cloud Program Manager,Program Management,project management
0,Google Cloud Program Manager,Program Management,marketing
0,Google Cloud Program Manager,Program Management,engineering
...,...,...,...
1246,Campus Security Manager,Real Estate & Workplace Services,security
1246,Campus Security Manager,Real Estate & Workplace Services,software
1247,Facilities Manager,Real Estate & Workplace Services,None
1248,Physical Security Manager,Real Estate & Workplace Services,security


In [496]:
myData2['Title'][]

1187    BOLD Intern, Summer 2018
1187    BOLD Intern, Summer 2018
Name: Title, dtype: object